## Configuration
_Initial steps to get the notebook ready to play nice with our repository. Do not delete this section._

Code formatting with [black](https://pypi.org/project/nb-black/).

In [1]:
%load_ext lab_black

In [2]:
import os
import pytz
import glob
import pathlib

this_dir = pathlib.Path(os.path.abspath(""))
data_dir = this_dir / "data"

In [3]:
import requests
import pandas as pd
import json
from datetime import datetime

## Download

Retrieve the page

In [4]:
url = "https://services1.arcgis.com/pWmBUdSlVpXStHU6/ArcGIS/rest/services/Riverside_County_COVID19_Cases_Public/FeatureServer/1/query?where=1%3D1&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelIntersects&resultType=none&distance=0.0&units=esriSRUnit_Meter&returnGeodetic=false&outFields=*&returnGeometry=true&returnCentroid=false&featureEncoding=esriDefault&multipatchOption=xyFootprint&maxAllowableOffset=&geometryPrecision=&outSR=&datumTransformation=&applyVCSProjection=false&returnIdsOnly=false&returnUniqueIdsOnly=false&returnCountOnly=false&returnExtentOnly=false&returnQueryGeometry=false&returnDistinctValues=false&cacheHint=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&having=&resultOffset=&resultRecordCount=&returnZ=false&returnM=false&returnExceededLimitFeatures=true&quantizationParameters=&sqlFormat=none&f=pjson&token="

In [5]:
r = requests.get(url)

In [6]:
data = r.json()

## Parse

In [7]:
dict_list = []

In [8]:
for item in data["features"]:
    d = dict(
        county="Riverside",
        area=item["attributes"]["NAME"],
        confirmed_cases=item["attributes"]["Point_Count"],
        deaths=item["attributes"]["SUM_Deceased"],
    )
    dict_list.append(d)

In [9]:
df = pd.DataFrame(dict_list)

Get timestamp

In [10]:
date_url = "https://services1.arcgis.com/pWmBUdSlVpXStHU6/ArcGIS/rest/services/Riverside_County_COVID19_Cases_Public/FeatureServer/1?f=json"
date_r = requests.get(date_url)
date_data = date_r.json()

In [11]:
timestamp = date_data["editingInfo"]["lastEditDate"]

KeyError: 'editingInfo'

In [12]:
timestamp = datetime.fromtimestamp((timestamp / 1000))

In [13]:
latest_date = pd.to_datetime(timestamp).date()

In [14]:
df["county_date"] = latest_date

Get ready for export

In [15]:
export_df = df[["county", "area", "confirmed_cases", "county_date"]]

## Vet

In [16]:
len(export_df)

64

In [17]:
try:
    assert not len(export_df) > 64
except AssertionError:
    raise AssertionError("Riverside County's scraper has more rows than before")

In [18]:
try:
    assert not len(export_df) < 64
except AssertionError:
    raise AssertionError("Riverside's scraper is missing rows")

## Export

Set date

In [19]:
tz = pytz.timezone("America/Los_Angeles")

In [20]:
today = datetime.now(tz).date()

In [21]:
slug = "riverside"

In [22]:
export_df.to_csv(data_dir / slug / f"{today}.csv", index=False)

## Combine

In [23]:
csv_list = [
    i
    for i in glob.glob(str(data_dir / slug / "*.csv"))
    if not str(i).endswith("timeseries.csv")
]

In [24]:
df_list = []
for csv in csv_list:
    if "manual" in csv:
        df = pd.read_csv(csv, parse_dates=["date"])
    else:
        file_date = csv.split("/")[-1].replace(".csv", "")
        df = pd.read_csv(csv, parse_dates=["county_date"])
        df["date"] = file_date
    df_list.append(df)

In [25]:
df = pd.concat(df_list).sort_values(["date", "area"])

In [26]:
df.to_csv(data_dir / slug / "timeseries.csv", index=False)